In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
os.environ["KERAS_BACKEND"] = "torch"
import keras
from datamodules import data_processors
import sys
sys.path.append("..")
print(sys.version)
from rusmodules.inverse import get_constants
from rusmodules.eigenvals import get_eigenvalues

3.13.1 (main, Dec  9 2024, 00:00:00) [GCC 14.2.1 20240912 (Red Hat 14.2.1-3)]


In [2]:
def C_error(C_pred, C_original):
    errors = dict()
    for key in C_pred.keys():
        errors[key] = (C_pred[key] - C_original[key])/C_original[key]
    #fin for 
    return errors
#fin función

def organize_matrix(dic_C):
    C = np.zeros((6,6))
    C_prim = np.array(tuple(map(lambda i: tuple(map(lambda j: 1 if i == j and i<3 else 0, range(6))), range(6)))) #Valores de C00, C11, C22
    C_sec = np.array(tuple(map(lambda i: tuple(map(lambda j: 1 if i == j and i >= 3 else 0, range(6))), range(6)))) #Valores de C33, C44, C55
    C_shear_prim = np.array(tuple(map(lambda i: tuple(map(lambda j: 1 if i != j and i<3 and j<3 else 0, range(6))), range(6)))) #Valores de C01, C02, C12
    C_shear_sec = np.array(tuple(map(lambda i: tuple(map(lambda j: 1 if i != j and i>=3 and j>=3 else 0, range(6))), range(6)))) #Valores de C34, C35, C45
    C_prim = C_prim * dic_C["C00"]
    C_sec = C_sec * dic_C["C33"]
    C_shear_prim = C_shear_prim * dic_C["C01"]
    C = C_prim + C_sec + C_shear_prim
    return C
#fin funcion

def verify_Cs(C_ori, eta, beta, model, N_eig = 20, include_x0 = True):
    C = organize_matrix(C_ori)
    eigs = get_eigenvalues(6, C, eta, beta, "Parallelepiped")["eig"][:N_eig]
    C_predicted = get_constants(eigs, eta, beta, model, include_x0)
    return C_predicted
#fin función

<p><b> Let's start testing with the ratio 3:4:5 as reported by Fukuda. That implies: </b></p>

*   $\eta = 2\arctan{\left(\frac{(3^2 + 4^2)^{1/2}}{5} \right)} = \frac{\pi}{2}$

*   $\beta = 4\arctan{\left(\frac{3}{4} \right)} = 0.82\pi$

Also, we need to update the data reported by Fukuda (as a pd.DataFrame called "rp_data"), excecute a forward problem for each row and finally execute the inverse problem importing the model to compare with the predicted values of Fukuda

In [3]:
eta = np.pi/2
beta = 4*np.arctan(3/4)
rp_data = pd.read_csv("../reference_data/Datos_Fukuda.csv")

In [4]:
rp_data

,Unnamed: 0,Ag,AgAl(98.4-1.6),AgAu(96-4),AgBr,AgCd(98.08-1.92),AgCl,AgIn(91.64-8.36),AgMg(96.93-3.07),AgSn(96.83-3.17),...,Th,ThO2,TlCl,UC,UO2,V,ZnS,ZnSe,ZnTe,ZrCo2
0,C11_Predicted(Gpa),114.00,129.90,127.60,47.3,123.1,50.00,107.3,115.3,124.6,...,73.4,350.5,47.7,324.6,371.8,235.20,101.50,75.50,76.60,223.50
1,C12_Predicted(Gpa),87.20,105.50,96.60,21.4,98.4,37.10,78.6,90.6,97.7,...,45.6,80.2,22.6,66.3,86.8,141.40,59.50,45.50,46.90,94.10
2,C44_Predicted(Gpa),46.20,48.40,46.20,6.9,49.6,14.50,40.8,46.9,49.1,...,44.2,63.4,7.5,58.6,64.2,47.20,42.90,44.90,30.40,79.10
3,E1_Predicted(Gpa),38.50,35.30,44.30,33.9,35.7,18.30,40.8,35.6,38.8,...,38.4,320.6,33.2,302.1,339.0,129.00,57.50,41.20,41.00,167.80
4,C11_Reported(Gpa),123.99,128.07,124.27,56.3,121.6,60.10,116.6,119.8,121.0,...,75.3,367.0,40.1,320.0,396.0,227.95,104.62,80.96,71.34,233.11
5,C12_Reported(Gpa),93.67,98.30,92.66,32.8,91.3,36.20,89.0,89.8,92.2,...,48.9,106.0,15.3,85.0,121.0,118.70,65.34,48.81,40.78,112.71
6,C44_Reported(Gpa),46.12,46.43,47.28,7.2,45.9,6.25,45.0,46.0,45.8,...,47.8,79.7,7.6,64.7,64.1,42.55,46.13,44.05,31.15,83.66
7,E1_Reported(Gpa),43.40,42.70,45.10,32.2,43.3,32.90,39.5,42.9,41.3,...,36.8,319.5,31.6,284.3,339.4,146.70,54.40,44.20,41.70,159.60
8,C11_Error(%),-8.10,1.50,2.70,-16.0,1.3,-16.80,-8.0,-3.7,3.0,...,-2.5,-4.5,19.0,1.4,-6.1,3.20,-3.00,-6.80,7.40,-4.10
9,C12_Error(%),-7.00,7.40,4.30,-34.7,7.8,2.50,-11.6,0.9,6.0,...,-6.7,-24.3,47.5,-22.0,-28.3,19.20,-8.90,-6.80,14.90,-16.50


In [5]:
exp_data = rp_data.iloc[4:8]
path_modelo = "models/cubico_L1.keras"
modelo = keras.models.load_model(path_modelo)

In [8]:
exp_data

,Unnamed: 0,Ag,AgAl(98.4-1.6),AgAu(96-4),AgBr,AgCd(98.08-1.92),AgCl,AgIn(91.64-8.36),AgMg(96.93-3.07),AgSn(96.83-3.17),...,Th,ThO2,TlCl,UC,UO2,V,ZnS,ZnSe,ZnTe,ZrCo2
4,C11_Reported(Gpa),123.99,128.07,124.27,56.3,121.6,60.10,116.6,119.8,121.0,...,75.3,367.0,40.1,320.0,396.0,227.95,104.62,80.96,71.34,233.11
5,C12_Reported(Gpa),93.67,98.30,92.66,32.8,91.3,36.20,89.0,89.8,92.2,...,48.9,106.0,15.3,85.0,121.0,118.70,65.34,48.81,40.78,112.71
6,C44_Reported(Gpa),46.12,46.43,47.28,7.2,45.9,6.25,45.0,46.0,45.8,...,47.8,79.7,7.6,64.7,64.1,42.55,46.13,44.05,31.15,83.66
7,E1_Reported(Gpa),43.40,42.70,45.10,32.2,43.3,32.90,39.5,42.9,41.3,...,36.8,319.5,31.6,284.3,339.4,146.70,54.40,44.20,41.70,159.60
